# BERT 전이학습

### 데이터 준비

In [ ]:
# 데이터 다운로드
from tensorflow.keras.utils import get_file

ratings_train_path = get_file('ratings_train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
ratings_test_path = get_file('ratings_test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

14628807/14628807 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4893335/4893335 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# 데이터 로드
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep='\t')
ratings_test_df = pd.read_csv(ratings_test_path, sep='\t')

display(ratings_train_df.head())
display(ratings_test_df.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
# 결측치 제거
ratings_train_df = ratings_train_df.dropna(how='any')
ratings_test_df = ratings_test_df.dropna(how='any')

display(ratings_train_df.info())
display(ratings_test_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


None

In [ ]:
# 샘플링
ratings_train_df = ratings_train_df.sample(n=15000, random_state=0)
ratings_test_df = ratings_test_df.sample(n=5000, random_state=0)

ratings_train_df['label'].value_counts(), ratings_test_df['label'].value_counts()

(label
 0    7512
 1    7488
 Name: count, dtype: int64,
 label
 0    2532
 1    2468
 Name: count, dtype: int64)

In [5]:
X_train = ratings_train_df['document'].values.tolist()
y_train = ratings_train_df['label'].values.tolist()

X_test = ratings_test_df['document'].values.tolist()
y_test = ratings_test_df['label'].values.tolist()

### 토커나이저/모델 준비

- bert 한국어 버전 사전학습 모델 klue/bert-base

In [6]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('klue/bert-base')
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# 토큰화
X_train = tokenizer(X_train, padding=True, truncation=True)
X_test = tokenizer(X_test, padding=True, truncation=True)

X_train[:3], X_test[:3]

([Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])],
 [Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])])

In [ ]:
print(X_train['input_ids'][0])          # input_ids: 단어 토큰화
print(X_train['attention_mask'][0])     # attention_mask: 실제값/패딩 구분
print(X_train['token_type_ids'][0])     # token_type_ids: 첫번째/두번째 문장 구분 (단일 문장이라 다 0)

[2, 1800, 2178, 860, 3629, 16516, 2031, 18, 18, 18, 14242, 2205, 2062, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 데이터 파이프라인 생성

In [9]:
type(X_train)

transformers.tokenization_utils_base.BatchEncoding

In [10]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))
test_ds = tf.data.Dataset.from_tensor_slices((dict(X_test), y_test))

In [11]:
train_dataset = train_ds.shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
test_dataset = test_ds.batch(64).prefetch(tf.data.AUTOTUNE)

### 모델 학습

In [12]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2, from_pt=True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should p

In [ ]:
from transformers import create_optimizer

num_train_steps = len(train_dataset) * 5
num_warmup_steps = int(num_train_steps * 0.1)

optimizer, _ = create_optimizer(
    init_lr=5e-5,                       # 학습률 초기값
    num_train_steps=num_train_steps,    # 총 학습 step수
    num_warmup_steps=num_warmup_steps,  # warmup step수 (총 step수 1/10)
    weight_decay_rate=0.1               # 학습률 감쇠 계수
)

model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

model.fit(train_dataset, epochs=5, validation_data=test_dataset, batch_size=64)

Epoch 1/5
235/235 [==============================] - 455s 2s/step - loss: 0.3947 - accuracy: 0.8189 - val_loss: 0.3333 - val_accuracy: 0.8588
Epoch 2/5
235/235 [==============================] - 434s 2s/step - loss: 0.2118 - accuracy: 0.9180 - val_loss: 0.3223 - val_accuracy: 0.8720
Epoch 3/5
235/235 [==============================] - 434s 2s/step - loss: 0.1067 - accuracy: 0.9627 - val_loss: 0.4253 - val_accuracy: 0.8762
Epoch 4/5
235/235 [==============================] - 434s 2s/step - loss: 0.0469 - accuracy: 0.9849 - val_loss: 0.5343 - val_accuracy: 0.8716
Epoch 5/5
235/235 [==============================] - 434s 2s/step - loss: 0.0274 - accuracy: 0.9916 - val_loss: 0.5586 - val_accuracy: 0.8726


### 모델 저장

In [14]:
model.save_pretrained('nsmc_model/bert-base')
tokenizer.save_pretrained('nsmc_model/bert-base')

('nsmc_model/bert-base/tokenizer_config.json',
 'nsmc_model/bert-base/special_tokens_map.json',
 'nsmc_model/bert-base/vocab.txt',
 'nsmc_model/bert-base/added_tokens.json',
 'nsmc_model/bert-base/tokenizer.json')

### 추론

In [ ]:
# 텍스트 분류 라벨 값 지정
model.config.id2label = {
    0: "부정",
    1: "긍정"
}

In [19]:
from transformers import TextClassificationPipeline

sentiment_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    return_all_scores=True
)

Device set to use 0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [20]:
sentiment_classifier('인생 영화를 찾았습니다!!!')

[[{'label': '부정', 'score': 0.0010393332922831178},
  {'label': '긍정', 'score': 0.9989606142044067}]]

# HuggingFace

### push

In [ ]:
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
REPO_NAME = 'bert-base-nsmc'

model.push_to_hub(REPO_NAME, use_temp_dir=True, use_auth_token=HF_TOKEN)
tokenizer.push_to_hub(REPO_NAME, use_temp_dir=True, use_auth_token=HF_TOKEN)

### load

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('squirreln/bert-base-nsmc')
model = AutoModel.from_pretrained('squirreln/bert-base-nsmc', from_tf=True)

In [ ]:
from transformers import pipeline

sentiment_classifier = pipeline('text-classification', model='squirreln/bert-base-nsmc')

In [ ]:
# 위 pipeline 코드에서 에러가 발생할 경우 대체
from transformers import pipeline, TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained('squirreln/bert-base-nsmc')

sentiment_classifier = pipeline(
    'text-classification', 
    model=model,
    tokenizer=tokenizer,
    framework='tf'
)

In [ ]:
# 추론
sentiment_classifier([
    '한국 영화는 이제 한물 갔어!',
    '어쩌라고 나는 재미있던데?ㅋㅋㅋ',
    '혜수 언니 사랑해요',
    '평생 잊지 못할 영화를 만났어요... 감동...'
])